# Wprowadzenie do sieci neuronowych i uczenia maszynowego - Sieci Rekurencyjne


---

**Prowadzący:** Piotr Baryczkowski, Jakub Bednarek<br>
**Kontakt:** piotr.baryczkowski@put.poznan.pl<br>

---

## Cel ćwiczeń:
- zapoznanie się z rekurencyjnymi sieciami neuronowymi,
- stworzenie modelu sieci z warstwami rekurencyjnymi dla zbioru danych MNIST,
- stworzenie własnych implementacji warstwami neuronowych

In [1]:
import numpy as np
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.5.1+cu121
CUDA available: True


In [2]:
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 9.91M/9.91M [00:02<00:00, 3.78MB/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 28.9k/28.9k [00:00<00:00, 134kB/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.26MB/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.95MB/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



## Sieci rekurencyjne
http://colah.github.io/posts/2015-08-Understanding-LSTMs/

https://pytorch.org/docs/stable/generated/torch.nn.RNN.html

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/

Przykładowy model z warstwą rekurencyjną dla danych MNIST:

In [4]:
class RecurrentModel(nn.Module):
    def __init__(self, num_classes=10):
        super(RecurrentModel, self).__init__()
        self.num_classes = num_classes
        # Define your layers here.
        self.lstm_1 = nn.LSTM(input_size=28, hidden_size=128, batch_first=True)
        self.relu_1 = nn.ReLU()
        self.dense_1 = nn.Linear(128, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, inputs):
        if inputs.dim() == 4:
            # Example: (batch_size, channels, sequence_length, features)
            inputs = inputs.squeeze(1)  # Remove the channels dimension if it's 1
        elif inputs.dim() != 3:
            raise ValueError(f"Expected input to be 3D, got {inputs.dim()}D instead.")

        lstm_out, _ = self.lstm_1(inputs)
        # Take the last output from the sequence (assume inputs are padded appropriately or have consistent lengths)
        x = lstm_out[:, -1, :]  # Get the output of the last time step
        x = self.relu_1(x)
        x = self.dense_1(x)
        return self.softmax(x)

model = RecurrentModel(num_classes=10)
model

RecurrentModel(
  (lstm_1): LSTM(28, 128, batch_first=True)
  (relu_1): ReLU()
  (dense_1): Linear(in_features=128, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)

In [5]:
learning_rate = 1e-3
batch_size = 32
epochs = 5

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302332  [   32/60000]
loss: 2.223289  [ 3232/60000]
loss: 2.079144  [ 6432/60000]
loss: 2.069159  [ 9632/60000]
loss: 1.823070  [12832/60000]
loss: 1.743898  [16032/60000]
loss: 1.810996  [19232/60000]
loss: 1.730737  [22432/60000]
loss: 1.747344  [25632/60000]
loss: 1.709906  [28832/60000]
loss: 1.566684  [32032/60000]
loss: 1.539778  [35232/60000]
loss: 1.671296  [38432/60000]
loss: 1.629454  [41632/60000]
loss: 1.781692  [44832/60000]
loss: 1.611585  [48032/60000]
loss: 1.616925  [51232/60000]
loss: 1.615725  [54432/60000]
loss: 1.524197  [57632/60000]
Test Error: 
 Accuracy: 90.0%, Avg loss: 1.565825 

Epoch 2
-------------------------------
loss: 1.580066  [   32/60000]
loss: 1.544541  [ 3232/60000]
loss: 1.595622  [ 6432/60000]
loss: 1.513075  [ 9632/60000]
loss: 1.532527  [12832/60000]
loss: 1.525020  [16032/60000]
loss: 1.530586  [19232/60000]
loss: 1.539494  [22432/60000]
loss: 1.475263  [25632/60000]
loss: 1.526074  [28832/60000

### Zadanie 1
Rozszerz model z powyższego przykładu o kolejną warstwę rekurencyjną przed gęstą warstwą wyjściową.

Standardowe sieci neuronowe generują jeden wynik na podstawie jednego inputu.
Natomiast sieci rekurencyjne przetwarzają dane sekwencyjnie, w każdym kroku łącząc wynik poprzedniego przetwarzania i aktualnego wejścia. Dlatego domyślnym wejściem sieci neuronowej jest tensor 3-wymiarowy ([batch_size,sequence_size,sample_size]).
Domyślnie warstwy rekurencyjne w PyTorchu zwracają sekwencje wyników wszystkich kroków przetwarzania dla warstwy rekurencyjnej. Jeśli chcesz zwrócić tylko wyniki ostatniego przetwarzania dla warstwy rekurencyjnej, musisz samemu to zaimplementować np. `x = lstm_out[:, -1, :]`.


In [7]:
class RecurrentModel2(nn.Module):
    def __init__(self, num_classes=10):
        super(RecurrentModel2, self).__init__()
        self.num_classes = num_classes
        # Define your layers here.
        self.lstm_1 = nn.LSTM(input_size=28, hidden_size=128, batch_first=True)
        self.relu_1 = nn.ReLU()
        self.lstm_2 = nn.LSTM(input_size=128, hidden_size=128, batch_first=True)
        self.dense_1 = nn.Linear(128, num_classes)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, inputs):
        if inputs.dim() == 4:
            # Example: (batch_size, channels, sequence_length, features)
            inputs = inputs.squeeze(1)  # Remove the channels dimension if it's 1
        elif inputs.dim() != 3:
            raise ValueError(f"Expected input to be 3D, got {inputs.dim()}D instead.")

        # TODO forward
        lstm_out_1, _ = self.lstm_1(inputs)
        lstm_out_1 = self.relu_1(lstm_out_1)
        lstm_out_2, _ = self.lstm_2(lstm_out_1)
        x = lstm_out_2[:, -1, :]
        x = self.relu_1(x)
        x = self.dense_1(x)
        return self.softmax(x)

model = RecurrentModel2(num_classes=10)
model

RecurrentModel2(
  (lstm_1): LSTM(28, 128, batch_first=True)
  (relu_1): ReLU()
  (lstm_2): LSTM(128, 128, batch_first=True)
  (dense_1): Linear(in_features=128, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)

In [8]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302471  [   32/60000]
loss: 2.085634  [ 3232/60000]
loss: 1.903280  [ 6432/60000]
loss: 1.699969  [ 9632/60000]
loss: 1.782170  [12832/60000]
loss: 1.629536  [16032/60000]
loss: 1.627146  [19232/60000]
loss: 1.638668  [22432/60000]
loss: 1.684394  [25632/60000]
loss: 1.502137  [28832/60000]
loss: 1.541847  [32032/60000]
loss: 1.536700  [35232/60000]
loss: 1.530181  [38432/60000]
loss: 1.507968  [41632/60000]
loss: 1.555423  [44832/60000]
loss: 1.512344  [48032/60000]
loss: 1.529061  [51232/60000]
loss: 1.568983  [54432/60000]
loss: 1.489966  [57632/60000]
Test Error: 
 Accuracy: 94.6%, Avg loss: 1.517058 

Epoch 2
-------------------------------
loss: 1.555691  [   32/60000]
loss: 1.559569  [ 3232/60000]
loss: 1.495042  [ 6432/60000]
loss: 1.516772  [ 9632/60000]
loss: 1.519558  [12832/60000]
loss: 1.487085  [16032/60000]
loss: 1.584513  [19232/60000]
loss: 1.599008  [22432/60000]
loss: 1.493178  [25632/60000]
loss: 1.497211  [28832/60000

### Zadanie 2
Wykorzystując model z przykładu, napisz sieć rekurencyjną przy użyciu RNNCell.

RNNCell implementuje tylko operacje wykonywane przez warstwę
rekurencyjną dla jednego kroku. Warstwy rekurencyjne w każdym kroku
łączą wynik operacji poprzedniego kroku i aktualny input.
Wykorzystaj pętle for do wielokrotnego wywołania komórki RNNCell (liczba kroków to liczba elementów w sekwencji).

Wywołanie zainicjalizowanej komórki rekurencyjnej wymaga podania aktualnego inputu i listy stanów ukrytych poprzedniego kroku (RNNCell ma jeden stan).

Trzeba zainicjalizować ukryty stan warstwy z wartościami początkowymi (można wykorzystać zmienne losowe - torch.rand).

In [9]:
import torch
import torch.nn as nn

class RecurrentModel3(nn.Module):
    def __init__(self, input_size=28, hidden_size=128, num_classes=10):
        super(RecurrentModel3, self).__init__()
        self.hidden_size = hidden_size
        self.num_classes = num_classes

        # Define the RNN cell
        self.rnn_cell = nn.RNNCell(input_size=input_size, hidden_size=hidden_size)
        self.relu = nn.ReLU()
        self.dense = nn.Linear(hidden_size, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, inputs):
        if inputs.dim() == 4:
            # Example: (batch_size, channels, sequence_length, features)
            inputs = inputs.squeeze(1)  # Remove the channels dimension if it's 1
        elif inputs.dim() != 3:
            raise ValueError(f"Expected input to be 3D, got {inputs.dim()}D instead.")

        # TODO forward
        batch_size, seq_len, _ = inputs.shape
        hidden = torch.zeros(batch_size, self.hidden_size, device=inputs.device)

        for i in range(seq_len):
            x_i = inputs[:, i, :]
            hidden = self.rnn_cell(x_i, hidden)

        x = self.relu(hidden)
        x = self.dense(x)

        return self.softmax(x)

model = RecurrentModel3(input_size=28, hidden_size=128, num_classes=10)


In [10]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303467  [   32/60000]
loss: 2.303347  [ 3232/60000]
loss: 2.290956  [ 6432/60000]
loss: 2.290200  [ 9632/60000]
loss: 2.265449  [12832/60000]
loss: 2.264529  [16032/60000]
loss: 2.218596  [19232/60000]
loss: 2.357947  [22432/60000]
loss: 2.200411  [25632/60000]
loss: 2.037620  [28832/60000]
loss: 2.098140  [32032/60000]
loss: 2.115752  [35232/60000]
loss: 2.005215  [38432/60000]
loss: 2.114975  [41632/60000]
loss: 2.023198  [44832/60000]
loss: 1.896647  [48032/60000]
loss: 1.894872  [51232/60000]
loss: 1.898192  [54432/60000]
loss: 1.767151  [57632/60000]
Test Error: 
 Accuracy: 64.5%, Avg loss: 1.830975 

Epoch 2
-------------------------------
loss: 1.852725  [   32/60000]
loss: 1.771270  [ 3232/60000]
loss: 1.920086  [ 6432/60000]
loss: 1.863017  [ 9632/60000]
loss: 1.930732  [12832/60000]
loss: 1.850483  [16032/60000]
loss: 1.669114  [19232/60000]
loss: 1.924139  [22432/60000]
loss: 1.786505  [25632/60000]
loss: 1.789915  [28832/60000

### Zadanie 3
Zamień komórkę rekurencyjną z poprzedniego zadania na LSTMCell.

In [8]:
class RecurrentModel4(nn.Module):
    def __init__(self, input_size=28, hidden_size=128, num_classes=10):
        super(RecurrentModel4, self).__init__()
        self.hidden_size = hidden_size
        self.num_classes = num_classes

        # Define the LSTM cell
        self.rnn_cell = nn.RNNCell(input_size=input_size, hidden_size=hidden_size)
        self.relu = nn.ReLU()
        self.dense = nn.Linear(hidden_size, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, inputs):
        if inputs.dim() == 4:
            # Example: (batch_size, channels, sequence_length, features)
            inputs = inputs.squeeze(1)  # Remove the channels dimension if it's 1
        elif inputs.dim() != 3:
            raise ValueError(f"Expected input to be 3D, got {inputs.dim()}D instead.")

        # TODO forward
        batch_size, seq_len, _ = inputs.shape
        hidden = torch.zeros(batch_size, self.hidden_size, device=inputs.device)

        for t in range(seq_len):
            x_t = inputs[:, t, :]
            hidden = self.rnn_cell(x_t, hidden)

        x = self.relu(hidden)
        x = self.dense(x)
        return self.softmax(x)

        return

model = RecurrentModel4(input_size=28, hidden_size=128, num_classes=10)
model

RecurrentModel4(
  (rnn_cell): RNNCell(28, 128)
  (relu): ReLU()
  (dense): Linear(in_features=128, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)

In [9]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302330  [   32/60000]
loss: 2.296053  [ 3232/60000]
loss: 2.272177  [ 6432/60000]
loss: 2.116109  [ 9632/60000]
loss: 2.046221  [12832/60000]
loss: 1.950159  [16032/60000]
loss: 1.833041  [19232/60000]
loss: 1.878635  [22432/60000]
loss: 1.741462  [25632/60000]
loss: 1.768229  [28832/60000]
loss: 1.872098  [32032/60000]
loss: 2.037668  [35232/60000]
loss: 1.791511  [38432/60000]
loss: 1.794800  [41632/60000]
loss: 1.820330  [44832/60000]
loss: 1.662935  [48032/60000]
loss: 1.836876  [51232/60000]
loss: 1.663052  [54432/60000]
loss: 1.743896  [57632/60000]
Test Error: 
 Accuracy: 73.7%, Avg loss: 1.733951 

Epoch 2
-------------------------------
loss: 1.745198  [   32/60000]
loss: 1.841552  [ 3232/60000]
loss: 1.812655  [ 6432/60000]
loss: 1.625731  [ 9632/60000]
loss: 1.698390  [12832/60000]
loss: 1.658336  [16032/60000]
loss: 1.625961  [19232/60000]
loss: 1.683396  [22432/60000]
loss: 1.602611  [25632/60000]
loss: 1.647871  [28832/60000

### Zadanie 4
Wykorzystując model z poprzedniego zadania, stwórz model sieci
neuronowej z własną implementacją prostej warstwy rekurencyjnej.
- w call zamień self.lstm_cell_layer(x) na wyołanie własnej metody np. self.cell(x)
- w konstruktorze modelu usuń inicjalizację komórki LSTM i zastąp ją inicjalizacją warstw potrzebnych do stworzenia własnej komórki rekurencyjnej,
- stwórz metodę cell() wykonującą operacje warstwy rekurencyjnej,
- prosta warstwa rekurencyjna konkatenuje poprzedni wyniki i aktualny input, a następnie przepuszcza ten połączony tensor przez warstwę gęstą (Dense).

In [11]:
class RecurrentModel5(nn.Module):
    def __init__(self, input_size=28, hidden_size=128, num_classes=10):
        super(RecurrentModel5, self).__init__()
        self.hidden_size = hidden_size
        self.num_classes = num_classes

        # Define the LSTM cell
        self.rnn_update = nn.Linear(input_size + hidden_size, hidden_size)
        self.activation = nn.ReLU()
        self.output_layer = nn.Linear(hidden_size, num_classes)

    def cell(self, x, h):
        combined = torch.cat([x, h], dim=1)
        h_next = self.rnn_update(combined)
        h_next = self.activation(h_next)
        return h_next

    def forward(self, inputs):
        if inputs.dim() == 4:
            # Example: (batch_size, channels, sequence_length, features)
            inputs = inputs.squeeze(1)  # Remove the channels dimension if it's 1
        elif inputs.dim() != 3:
            raise ValueError(f"Expected input to be 3D, got {inputs.dim()}D instead.")

        # TODO forward
        batch_size, seq_len, _ = inputs.shape
        h = torch.zeros(batch_size, self.hidden_size, device=inputs.device)

        for t in range(seq_len):
            x_t = inputs[:, t, :]
            h = self.cell(x_t, h)

        logits = self.output_layer(h)
        return logits

model = RecurrentModel5(input_size=28, hidden_size=128, num_classes=10)
model

RecurrentModel5(
  (rnn_update): Linear(in_features=156, out_features=128, bias=True)
  (activation): ReLU()
  (output_layer): Linear(in_features=128, out_features=10, bias=True)
)

In [12]:
epochs = 5

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.310385  [   32/60000]
loss: 1.396334  [ 3232/60000]
loss: 1.498438  [ 6432/60000]
loss: 1.256120  [ 9632/60000]
loss: 0.932033  [12832/60000]
loss: 0.488258  [16032/60000]
loss: 0.591896  [19232/60000]
loss: 0.578803  [22432/60000]
loss: 0.613620  [25632/60000]
loss: 0.940471  [28832/60000]
loss: 0.422193  [32032/60000]
loss: 0.738239  [35232/60000]
loss: 0.678760  [38432/60000]
loss: 0.280115  [41632/60000]
loss: 0.818911  [44832/60000]
loss: 0.954295  [48032/60000]
loss: 0.347573  [51232/60000]
loss: 0.228120  [54432/60000]
loss: 0.452766  [57632/60000]
Test Error: 
 Accuracy: 89.7%, Avg loss: 0.342248 

Epoch 2
-------------------------------
loss: 0.261048  [   32/60000]
loss: 0.190358  [ 3232/60000]
loss: 0.233266  [ 6432/60000]
loss: 0.680700  [ 9632/60000]
loss: 0.390971  [12832/60000]
loss: 0.133683  [16032/60000]
loss: 0.616845  [19232/60000]
loss: 0.275304  [22432/60000]
loss: 0.789806  [25632/60000]
loss: 0.292102  [28832/60000

### Zadanie 5

Na podstawie modelu z poprzedniego zadania stwórz model z własną implementacją warstwy LSTM. Dokładny i zrozumiały opis działania wartswy LSTM znajduje się na [stronie](http://colah.github.io/posts/2015-08-Understanding-LSTMs/).

In [13]:
from torch.nn.modules.activation import Sigmoid, Tanh

class RecurrentModel6(nn.Module):
    def __init__(self, input_size=28, hidden_size=128, num_classes=10):
        super(RecurrentModel6, self).__init__()
        self.hidden_size = hidden_size
        self.num_classes = num_classes

        # Define LSTM layers
        self.Wf = nn.Linear(input_size + hidden_size, hidden_size)
        self.Wi = nn.Linear(input_size + hidden_size, hidden_size)
        self.Wo = nn.Linear(input_size + hidden_size, hidden_size)
        self.Wg = nn.Linear(input_size + hidden_size, hidden_size)

        self.sigmoid = Sigmoid()
        self.tanh = Tanh()

        self.output_layer = nn.Linear(hidden_size, num_classes)

    def cell(self, x, h, c):
        combined = torch.cat([x, h], dim=1)  # shape: [batch_size, input_size + hidden_size]

        # Wyliczamy bramki
        f_t = self.sigmoid(self.Wf(combined))
        i_t = self.sigmoid(self.Wi(combined))
        o_t = self.sigmoid(self.Wo(combined))
        g_t = self.tanh(self.Wg(combined))

        # Aktualizujemy stan komórki
        c_next = f_t * c + i_t * g_t
        # Aktualizujemy stan ukryty
        h_next = o_t * self.tanh(c_next)

        return h_next, c_next

    def forward(self, inputs):
        if inputs.dim() == 4:
            # Example: (batch_size, channels, sequence_length, features)
            inputs = inputs.squeeze(1)  # Remove the channels dimension if it's 1
        elif inputs.dim() != 3:
            raise ValueError(f"Expected input to be 3D, got {inputs.dim()}D instead.")

        batch_size, seq_len, _ = inputs.shape
        h = torch.zeros(batch_size, self.hidden_size, device=inputs.device)
        c = torch.zeros(batch_size, self.hidden_size, device=inputs.device)

        for t in range(seq_len):
            x_t = inputs[:, t, :]
            h, c = self.cell(x_t, h, c)

        logits = self.output_layer(h)
        return logits

model = RecurrentModel6(input_size=28, hidden_size=128, num_classes=10)
model

RecurrentModel6(
  (Wf): Linear(in_features=156, out_features=128, bias=True)
  (Wi): Linear(in_features=156, out_features=128, bias=True)
  (Wo): Linear(in_features=156, out_features=128, bias=True)
  (Wg): Linear(in_features=156, out_features=128, bias=True)
  (sigmoid): Sigmoid()
  (tanh): Tanh()
  (output_layer): Linear(in_features=128, out_features=10, bias=True)
)

In [14]:
epochs = 2
learning_rate = 0.001
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301794  [   32/60000]
loss: 1.151685  [ 3232/60000]
loss: 0.939542  [ 6432/60000]
loss: 0.374727  [ 9632/60000]
loss: 0.515072  [12832/60000]
loss: 0.665575  [16032/60000]
loss: 0.381911  [19232/60000]
loss: 0.583883  [22432/60000]
loss: 0.343167  [25632/60000]
loss: 0.107487  [28832/60000]
loss: 0.648047  [32032/60000]
loss: 0.189443  [35232/60000]
loss: 0.578680  [38432/60000]
loss: 0.545597  [41632/60000]
loss: 0.687089  [44832/60000]
loss: 0.100544  [48032/60000]
loss: 0.258194  [51232/60000]
loss: 0.055723  [54432/60000]
loss: 0.164558  [57632/60000]
Test Error: 
 Accuracy: 95.1%, Avg loss: 0.165891 

Epoch 2
-------------------------------
loss: 0.254426  [   32/60000]
loss: 0.087994  [ 3232/60000]
loss: 0.251491  [ 6432/60000]
loss: 0.178177  [ 9632/60000]
loss: 0.241674  [12832/60000]
loss: 0.068503  [16032/60000]
loss: 0.107146  [19232/60000]
loss: 0.208905  [22432/60000]
loss: 0.218979  [25632/60000]
loss: 0.048008  [28832/60000